# 生成华大版本的bgef

In [2]:
%%writefile gem2gef_bgi_bug.py
from gefpy import bgef_writer_cy
bgef_writer_cy.generate_bgef(
    input_file = "out/Y00855N1.tissue.gem.gz",
    bgef_file = "out/Y00855N1_bgi.bgef",
    bin_sizes = [1],
)

Overwriting gem2gef_bgi_bug.py


上述代码会报错
> (st) PS D:\Rust\geftools_rs> & E:/Anaconda_envs/envs/st/python.exe d:/Rust/geftools_rs/gem2gef_bgi_bug.py \
geneID  x       y       MIDCount        ExonCount\
 5\
create bgef file: out/Y00855N1_bgi.bgef\
can not find sn info from input file out/Y00855N1.tissue.gem.gz\
bin 1 matrix: min_x=0 len_x=-2147483648 min_y=0 len_y=-2147483648 matrix_len=0

In [3]:
%%writefile gem2gef_bgi_bug.py
from gefpy import bgef_writer_cy
bgef_writer_cy.generate_bgef(
    input_file = "out/test10000.gem.gz",
    bgef_file = "out/test10000_bgi.bgef",
    bin_sizes = [1],
)

Overwriting gem2gef_bgi_bug.py


In [7]:
import os
import h5py
import numpy as np
display(os.getcwd())

def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"[Group] {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"[Dataset] {name}  shape={obj.shape}  dtype={obj.dtype}")

path = "out/test10000_bgi.bgef"
with h5py.File(path, "r") as f:
    f.visititems(print_structure)

    print("\nRoot attributes:")
    for k, v in f.attrs.items():
        # bytes → 字符串
        if isinstance(v, bytes):
            v = v.decode()
        # numpy 标量 → 提取值但保留 dtype
        if isinstance(v, np.generic):
            print(f"  {k}: {v.item()} (dtype={v.dtype})")
        else:
            print(f"  {k}: {v} (type={type(v).__name__})")

'd:\\Rust\\geftools_rs'

[Group] geneExp
[Group] geneExp/bin1
[Dataset] geneExp/bin1/exon  shape=(8325,)  dtype=uint16
[Dataset] geneExp/bin1/expression  shape=(8325,)  dtype=[('x', '<i4'), ('y', '<i4'), ('count', '<u2')]
[Dataset] geneExp/bin1/gene  shape=(3051,)  dtype=[('geneID', 'S64'), ('geneName', 'S64'), ('offset', '<u4'), ('count', '<u4')]
[Group] stat
[Dataset] stat/gene  shape=(3051,)  dtype=[('geneID', 'S64'), ('geneName', 'S64'), ('MIDcount', '<u4'), ('E10', '<f4')]
[Group] wholeExp
[Dataset] wholeExp/bin1  shape=(21270, 21263)  dtype=[('MIDcount', '<u4'), ('genecount', '<u2')]
[Group] wholeExpExon
[Dataset] wholeExpExon/bin1  shape=(21270, 21263)  dtype=uint32

Root attributes:
  bin_type: [b'Bin'] (type=ndarray)
  gef_area: [0.] (type=ndarray)
  geftool_ver: [ 1  1 20] (type=ndarray)
  omics: [b'Transcriptomics'] (type=ndarray)
  version: [4] (type=ndarray)


In [9]:
# 代码示例
import h5py
import numpy as np

path = "out/Y00855N1.bgef"
with h5py.File(path, "r") as f:
    h5path = "wholeExp/bin1"

    # 输出属性
    print(">>>Attribute:")
    for k, v in f[h5path].attrs.items():
        # 一些属性是 bytes，需要手动 decode
        if isinstance(v, bytes):
            v = v.decode()
        print(f"{k}: {v}, {v.dtype}")
    
    # 数据矩阵
    print(">>>Dataset:")
    expr = np.array(f[h5path]) # 替换这里f的键值即可
    print(expr.shape, expr)

>>>Attribute:
lenX: 17456, int32
lenY: 18054, int32
maxExon: 25, uint32
maxGene: 47, uint32
maxMID: 49, uint32
minX: 3767, int32
minY: 3330, int32
number: 85019725, uint64
resolution: 500, uint32
>>>Dataset:
(17456, 18054) [[(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 ...
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(1, 1) (2, 2) (0, 0) ... (3, 3) (4, 3) (1, 1)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (1, 1) (0, 0)]]


# 查看输出hdf5数据结构

In [15]:
import os
import h5py
display(os.getcwd())

def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"[Group] {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"[Dataset] {name}  shape={obj.shape}  dtype={obj.dtype}")

path = "out/Y00855N1.bgef"
with h5py.File(path, "r") as f:
    f.visititems(print_structure)

    print("\nRoot attributes:")
    for k, v in f.attrs.items():
        # bytes → 字符串
        if isinstance(v, bytes):
            v = v.decode()
        # numpy 标量 → 提取值但保留 dtype
        if isinstance(v, np.generic):
            print(f"  {k}: {v.item()} (dtype={v.dtype})")
        else:
            print(f"  {k}: {v} (type={type(v).__name__})")

'd:\\Rust\\geftools_rs'

[Group] geneExp
[Group] geneExp/bin1
[Dataset] geneExp/bin1/exon  shape=(153549346,)  dtype=uint32
[Dataset] geneExp/bin1/expression  shape=(153549346,)  dtype=[('x', '<i4'), ('y', '<i4'), ('count', '<u4')]
[Dataset] geneExp/bin1/gene  shape=(40417,)  dtype=[('geneID', 'S64'), ('geneName', 'S64'), ('offset', '<u4'), ('count', '<u4')]
[Group] wholeExp
[Dataset] wholeExp/bin1  shape=(17456, 18054)  dtype={'names': ['MIDcount', 'genecount'], 'formats': ['<u4', '<u2'], 'offsets': [0, 4], 'itemsize': 8}
[Group] wholeExpExon
[Dataset] wholeExpExon/bin1  shape=(17456, 18054)  dtype=uint32

Root attributes:
  bin_type: Bin (type=str)
  gef_area: 0.0 (dtype=float32)
  geftool_ver: [ 1  1 20] (type=ndarray)
  omics: Transcriptomics (type=str)
  sn: Y00855N1 (type=str)
  version: 4 (dtype=uint32)


Y00855ND.tissue.gem.gz文件总行数是123196419

In [13]:
# 代码示例
import h5py
import numpy as np
path = "out/test.bgef"
path = "out/Y00855N1.bgef"
with h5py.File(path, "r") as f:
    # h5path = "/geneExp/bin1/expression"
    h5path = "wholeExp/bin1"

    # 输出属性
    print(">>>Attribute:")
    for k, v in f[h5path].attrs.items():
        # 一些属性是 bytes，需要手动 decode
        if isinstance(v, bytes):
            v = v.decode()
        print(f"{k}: {v}, {v.dtype}")
    
    # 数据矩阵
    print(">>>Dataset:")
    expr = np.array(f[h5path]) # 替换这里f的键值即可
    print(expr.shape, expr)

>>>Attribute:
lenX: 17456, int32
lenY: 18054, int32
maxExon: 25, uint32
maxGene: 47, uint32
maxMID: 49, uint32
minX: 3767, int32
minY: 3330, int32
number: 85019725, uint64
resolution: 500, uint32
>>>Dataset:
(17456, 18054) [[(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 ...
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (0, 0) (0, 0)]
 [(1, 1) (2, 2) (0, 0) ... (3, 3) (4, 3) (1, 1)]
 [(0, 0) (0, 0) (0, 0) ... (0, 0) (1, 1) (0, 0)]]


# 查看cgef结构

In [16]:
import os
import h5py
display(os.getcwd())

def print_structure(name, obj):
    if isinstance(obj, h5py.Group):
        print(f"[Group] {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"[Dataset] {name}  shape={obj.shape}  dtype={obj.dtype}")

path = "out/cell_cut_out/Y00855N1.cellbin.gef"
with h5py.File(path, "r") as f:
    f.visititems(print_structure)

    print("\nRoot attributes:")
    for k, v in f.attrs.items():
        # bytes → 字符串
        if isinstance(v, bytes):
            v = v.decode()
        # numpy 标量 → 提取值但保留 dtype
        if isinstance(v, np.generic):
            print(f"  {k}: {v.item()} (dtype={v.dtype})")
        else:
            print(f"  {k}: {v} (type={type(v).__name__})")

'd:\\Rust\\geftools_rs'

[Group] cellBin
[Dataset] cellBin/blockIndex  shape=(4900,)  dtype=uint32
[Dataset] cellBin/blockSize  shape=(4,)  dtype=uint32
[Dataset] cellBin/cell  shape=(98651,)  dtype=[('id', '<u4'), ('x', '<i4'), ('y', '<i4'), ('offset', '<u4'), ('geneCount', '<u2'), ('expCount', '<u2'), ('dnbCount', '<u2'), ('area', '<u2'), ('cellTypeID', '<u2'), ('clusterID', '<u2')]
[Dataset] cellBin/cellBorder  shape=(98651, 32, 2)  dtype=int16
[Dataset] cellBin/cellExon  shape=(98651,)  dtype=uint16
[Dataset] cellBin/cellExp  shape=(18658002,)  dtype=[('geneID', '<u4'), ('count', '<u2')]
[Dataset] cellBin/cellExpExon  shape=(18658002,)  dtype=uint16
[Dataset] cellBin/cellTypeList  shape=(1,)  dtype=|S32
[Dataset] cellBin/gene  shape=(40417,)  dtype=[('geneID', 'S64'), ('geneName', 'S64'), ('offset', '<u4'), ('cellCount', '<u4'), ('expCount', '<u4'), ('maxMIDcount', '<u2')]
[Dataset] cellBin/geneExon  shape=(40417,)  dtype=uint32
[Dataset] cellBin/geneExp  shape=(18658002,)  dtype=[('cellID', '<u4'), ('coun

In [17]:
# 代码示例
import h5py
import numpy as np

path = "out/cell_cut_out/Y00855N1.cellbin.gef"
with h5py.File(path, "r") as f:
    h5path = "cellBin/gene"

    # 输出属性
    print(">>>Attribute:")
    for k, v in f[h5path].attrs.items():
        # 一些属性是 bytes，需要手动 decode
        if isinstance(v, bytes):
            v = v.decode()
        print(f"{k}: {v}, {v.dtype}")
    
    # 数据矩阵
    print(">>>Dataset:")
    expr = np.array(f[h5path]) # 替换这里f的键值即可
    print(expr.shape, expr)

>>>Attribute:
maxCellCount: [77376], uint32
maxExpCount: [65272], uint32
minCellCount: [1], uint32
minExpCount: [1], uint32
>>>Dataset:
(40417,) [(b'0610005C13Rik', b'0610005C13Rik',        0,  61,  66, 2)
 (b'0610006L08Rik', b'0610006L08Rik',       61,  38,  38, 1)
 (b'0610009B22Rik', b'0610009B22Rik',       99, 479, 513, 3) ...
 (b'n-R5s97', b'n-R5s97',        0,   0,   0, 0)
 (b'n-R5s98', b'n-R5s98', 18658001,   1,   1, 1)
 (b'n-TSaga9', b'n-TSaga9',        0,   0,   0, 0)]


# 使用stereopy读取和显示

In [ ]:
import sys
import os
import pandas as pd
from natsort import natsorted
import stereo as st
from stereo.core.ms_data import MSData
from stereo.core.ms_pipeline import slice_generator
import warnings
warnings.filterwarnings('ignore')

targets = pd.read_csv("data/targets.csv")
# construct MSData object
ms_data = MSData(_relationship='other', _var_type='intersect')

for idx, row in targets.iterrows():
    print(row['acc_id'])
    # read the gem file
    data_path = f"data/cell_cut_out/{row['acc_id']}.cellbin.gef"
    # 原始GEM没有cellbin这一项，所以只能按bins读取
    ms_data += st.io.read_gef(
        file_path=data_path,
        bin_type='cell_bins',
        is_sparse=True,
        # gene_name_index=True, # 手搓的bgef没有真正的geneID
        )